In [ ]:
# CLAP Score compare
import numpy as np
import librosa
import torch
import CLAP.src.laion_clap as laion_clap

# quantization
def int16_to_float32(x):
    return (x / 32767.0).astype(np.float32)


def float32_to_int16(x):
    x = np.clip(x, a_min=-1., a_max=1.)
    return (x * 32767.).astype(np.int16)

model = laion_clap.CLAP_Module(enable_fusion=False)

In [ ]:
model.load_ckpt(model_id=1)

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import os

def cosine_distance(vector1, vector2):
    similarity = cosine_similarity(vector1.reshape(1, -1), vector2.reshape(1, -1))
    distance = 1 - similarity[0, 0]
    return distance
    
def euclidean_distance(tensor1, tensor2):
    distance = np.linalg.norm(tensor1 - tensor2)
    return distance

def get_distances(path):
    eu_dis, cos_dis, count = 0, 0, 0
    
    folders = os.listdir(path)
    for folder in tqdm(folders):
        files = os.listdir(f"{test_path}/{folder}")
        audio_paths = [f"{test_path}/{folder}/{ffp}" for ffp in files]
        
        audio_embed = model.get_audio_embedding_from_filelist(x = audio_paths)
        text_embed = model.get_text_embedding([folder for _ in range(len(files)+1)])
    
        for i in range(len(files)):
            eu_dis += euclidean_distance(audio_embed[i], text_embed[i])
            cos_dis += cosine_distance(audio_embed[i], text_embed[i])
            count += 1
    
    print("Average Euclidean distance : ", eu_dis/count)
    print("Average Cosine distance : ", cos_dis/count)

In [ ]:
test_path = "./test_base_38"
get_distances(test_path)

test_path = "./test_total_compare_38"
get_distances(test_path)

In [ ]:
# compare 1
import os
from audiotools import AudioSignal

folder_path = "./test_base_38/"
folders = os.listdir(folder_path)

for folder in folders[-7:]:
    print(folder)
    files = os.listdir(folder_path+folder)
    for file in files:
        AudioSignal(folder_path+folder+"/"+file).widget()